# Import modules
CDC data set description link [here](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Questionnaire&CycleBeginYear=2015)

In [154]:
import pdb
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold, SelectFromModel, SelectKBest, mutual_info_classif, mutual_info_regression
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

import nhanes as nhanes

%matplotlib notebook

## Settings

In [6]:
DATA_PATH = 'C:/Users/allen/Documents/Git-Repos/Opportunistic/CDC/NHANES/'
DATASET = 'cancer'

### Note: 
The code below loads each dataset: dataset_features, dataset_targets

Here, all datasets are defined explicitly (see nhanes.py).

In [7]:
ds = nhanes.Dataset(DATA_PATH)
ds.load_cancer()
n_fe = ds.features.shape[1]
n_classes = 2

Processing: Dietary\DR2TOT_H.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

Processing: Laboratory\PAH_F.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

Processing: Questionnaire\WHQ_H.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Extract the features and targets.

In [39]:
# indx = np.argwhere(ds.targets != 3)
dataset_features = ds.features # [ds.targets != 3]
dataset_targets = ds.targets # [ds.targets != 3]

# Pre-fix
# dataset_features = ds.features
# dataset_targets = ds.targets

## Preprocessing of Data
### Drop features with too-low variance

In [40]:
dataset_features_sel = dataset_features.copy()

# var_thresh = .001
# dropped_keys = []
# for key, value in dataset_features_sel.iteritems():
#     if(value.var() < var_thresh):
#         dataset_features_sel.drop(key, axis=1, inplace=True)
#         dropped_keys.append(key)

# print("Dropped %i keys, they were:\n %s" %(len(dropped_keys), dropped_keys))

### Seperate one-hot encoded, discrete, and continuous features

In [42]:
# onehot encoded
dataset_features_onehot = dataset_features_sel.filter(regex=(".*#.*"))
# Continuous 
dataset_features_cont = dataset_features_sel.drop(columns=dataset_features_sel.filter(regex=(".*#.*")))

dataset_features_disc = []
for key, value in dataset_features_cont.iteritems():
    # discrete were normalized so they should have a mean of .5, and we expect 3 of them
    if(value.mean() >= .5):
        dataset_features_disc.append(dataset_features_cont[key])
        dataset_features_cont.drop(key, axis=1, inplace=True)
# discrete
dataset_features_disc = pd.concat(dataset_features_disc, axis=1)
print(dataset_features_sel.shape)
print(dataset_features_cont.shape)
print(dataset_features_onehot.shape)
print(dataset_features_disc.shape)

(49509, 119)
(49509, 55)
(49509, 61)
(49509, 3)


### Calculate PCA to drop features

In [166]:
dfs = [dataset_features_cont.copy(), dataset_features_disc.copy(), dataset_features_onehot.copy()]
pca_dfs = []
for df in dfs:    
    pca = PCA() # no args, so keep all columns, n_components=2
    pca.fit(df)
    var_ratio = pca.explained_variance_ratio_
    var_ratio = var_ratio.reshape((1, len(var_ratio)))
    pca_dfs.append(pd.DataFrame(data=var_ratio, columns=df.columns))

for df in pca_dfs:
    print(df.shape)

(1, 55)
(1, 3)
(1, 61)


### Remove columns that account for too little of the variance

In [169]:
dfs_out = []

for i in range(len(dfs)):
    mi = pca_dfs[i] # mutual information dataframe
    df = dfs[i] # data containing dataframe
    to_keep = []
    var_threshold = 0.80
    # mi = mi.sort_values(by =0, axis=1) #sorts the values ascending
    # print(mi)
    var_counter = 0
    for col in mi:
        var_counter += mi[col].iloc[0]
        to_keep.append(col)
        if(var_counter > var_threshold):
            break
    if(len(to_keep) > 0):
        dfs_out.append(pd.DataFrame(df[to_keep].copy()))

for df in dfs_out:
    print(df.shape)

(49509, 32)
(49509, 2)
(49509, 11)


### Calculate mutual information
Very long computation, proceed with care

In [171]:
dataset_features_sel_vals = dataset_features_sel.values
dataset_targets_vals = np.ravel(dataset_targets.values) # unroll the column to the correct vector shape
# dfs = [dataset_features_cont.copy(), dataset_features_disc.copy(), dataset_features_onehot.copy()]
# retrieve values from dfs_out for subsequent steps
dataset_features_cont = dfs_out[0]
dataset_features_disc = dfs_out[1]
dataset_features_onehot = dfs_out[2]

mutual_info_cont = mutual_info_regression(dataset_features_cont.values, dataset_targets_vals)
mutual_info_disc = mutual_info_regression(dataset_features_disc.values, dataset_targets_vals)
mutual_info_onehot = mutual_info_classif(dataset_features_onehot.values, dataset_targets_vals)
# print(mutual_info)

c:\users\allen\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
c:\users\allen\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


### Convert mutual info into dataframes and inspect

In [182]:
if(mutual_info_cont.shape[0] != 1): # we only need to run this portion of the block once
    mutual_info_cont = mutual_info_cont.reshape((1, len(mutual_info_cont)))
    mutual_info_disc = mutual_info_disc.reshape((1, len(mutual_info_disc))) 
    mutual_info_onehot = mutual_info_onehot.reshape((1, len(mutual_info_onehot))) 

mutual_info_cont_df = pd.DataFrame(data=mutual_info_cont, columns=dataset_features_cont.columns)
mutual_info_disc_df = pd.DataFrame(data=mutual_info_disc, columns=dataset_features_disc.columns)
mutual_info_onehot_df = pd.DataFrame(data=mutual_info_onehot, columns=dataset_features_onehot.columns)

### Dropping features with low mutual information
    __Conditioned on target variable__

In [153]:
mi_dfs = [mutual_info_cont_df, mutual_info_disc_df, mutual_info_onehot_df]
dfs = [dataset_features_cont, dataset_features_disc, dataset_features_onehot]
dfs_out = []

for i in range(len(dfs)):
    mi = mi_dfs[i] # mutual information dataframe
    df = dfs[i] # data containing dataframe
    to_drop = []
    mean = mi.mean(axis=1)
    
    # mi = mi.sort_values(by =0, axis=1) #sorts the values ascending
    # print(mi)
    for col in mi:
        if(mi[col].iloc[0] < mean).all():
            to_drop.append(col)
    if(len(to_drop) > 0):
        dfs_out.append(pd.DataFrame(df.drop(columns=to_drop)))
    
dfs_out = pd.concat(dfs_out, axis=1)
# Retained columns
print(dfs_out.columns)
# output shape
print(dfs_out.shape)

Index(['INDFMIN2', 'INDFMPIR', 'DR2TKCAL', 'BPXPLS', 'URXUCD', 'URXUCO',
       'URXUCS', 'URXUSN', 'URXUSR', 'INDFMMPI', 'IND310', 'DBD895', 'CBQ585',
       'ALQ130', 'PAQ677', 'SMQ905', 'SMD480', 'LBXTC', 'ALQ101', 'ALQ120Q',
       'RIDAGEYR', 'RIDRETH3#1.0', 'RIDRETH3#3.0', 'RIDRETH3#6.0',
       'RIDRETH1#1.0', 'RIDRETH1#3.0', 'RIDRETH1#4.0', 'RIDRETH1#5.0',
       'BPXPULS#1.0', 'BPXPULS#2.0', 'MCQ080#2.0', 'MCQ365A#1.0',
       'MCQ365A#9.0', 'MCQ365B#1.0', 'MCQ370A#7.0', 'MCQ370B#1.0',
       'SLQ050#2.0', 'SMQ020#1.0', 'SMQ020#2.0'],
      dtype='object')
(49509, 39)


## Train/Test Separation

In [146]:
perm = np.random.permutation(dataset_targets.shape[0])
dataset_features = dfs_out.values[perm]
dataset_targets = dataset_targets_vals[perm]

print("dataset_features Shape: %s, dataset_targets Shape: %s" % (dataset_features.shape, dataset_targets.shape))

def get_batch(n_size, phase):
    # select indices
    n_samples = dataset_features.shape[0]
    n_classes = int(dataset_targets.max() + 1)
    if phase == 'test':
        inds_sel = np.arange(0, int(n_samples*0.15), 1)
    elif phase == 'validation':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.15), int(n_samples*0.30), 1)
    elif phase == 'train':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.30), n_samples, 1)
    else:
        raise NotImplementedError
    inds_sel = np.random.permutation(inds_sel)
    batch_inds = []
    for cl in range(n_classes):
        inds_cl = inds_sel[dataset_targets[inds_sel] == cl]
        batch_inds.extend(inds_cl[:n_size//n_classes])
    batch_inds = np.random.permutation(batch_inds)
    
    return dataset_features[batch_inds], dataset_targets[batch_inds]
    
features_trn, targets_trn = get_batch(n_size=5000, phase='train')
features_tst, targets_tst = get_batch(n_size=1000, phase='test')

dataset_features Shape: (49509, 39), dataset_targets Shape: (49509,)


## Classification

In [147]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_RFC', accu)

clf = SVC(gamma='auto')
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_SVC', accu)

clf = LogisticRegression(solver='lbfgs', max_iter=200)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_LR', accu)


accu_tst_RFC 0.6772277227722773
accu_tst_SVC 0.693069306930693


c:\users\allen\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accu_tst_LR 0.700990099009901


In [148]:
print(classification_report(targets_tst, preds_tst))

              precision    recall  f1-score   support

           0       0.69      0.75      0.72       250
           1       0.71      0.67      0.69       250
           3       0.00      0.00      0.00         5

   micro avg       0.70      0.70      0.70       505
   macro avg       0.47      0.47      0.47       505
weighted avg       0.69      0.70      0.70       505



c:\users\allen\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Accuracies from baseline: 
#### Cancer (ds.load_cancer()):
* accu_tst_RFC 0.758
* accu_tst_SVC 0.759
* accu_tst_LR 0.768